# In-class exercise 7: Deep Learning 1 (Part A)
In this notebook we will see how to write efficient and numerically stable code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import minmax_scale

## Loading the data

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

# Scale each feature to [-1, 1] range
X = minmax_scale(X, feature_range=(-1, 1))

Check the shapes

In [ ]:
# TODO

# 1. Vectorization

## 1.1. Logistic regression (two classes)

**Setting:** Logistic regression (two classes)

**Task:** Generate predictions for the entire dataset

**Data:** $X \in \mathbb{R}^{n \times d}$, $y \in \mathbb{R}^{n}$

**Model:** $f(x) = \sigma(w^T x + b)$

In [ ]:
n_features = X.shape[1]
w = np.random.normal(size=[n_features], scale=0.1)  # weight vector
b = np.random.normal(size=[1])  # bias

Check the shapes

In [ ]:
# TODO

Define the `sigmoid` function

In [ ]:
def sigmoid(t):
    """Apply sigmoid to the input array."""
    # TODO
    pass

Does it work for any input?

In [ ]:
# input is a scalar
print(sigmoid(0))

# input is a vector
print(sigmoid(np.array([0, 1, 2])))

# input is a matrix
print(sigmoid(np.array([[0, 1, 2], [-1, -2, -3]])))

This is called **broadcasting**. The smaller array is "broadcast" across the larger array so that they have compatible shapes. Numpy does this automatically. Let's see how it works. (Also see [here](https://numpy.org/doc/stable/user/basics.broadcasting.html#).)

In [ ]:
# How does broadcasting work between a scalar and a vector?
# TODO

In [ ]:
# How does broadcasting work between a scalar and a matrix?
# TODO

In [ ]:
# How does broadcasting work between a vector and a matrix?
# TODO

### Bad - for loops

Generate predictions with a logistic regression model using a for-loop.

In [ ]:
def predict_for_loop(X, w, b):
    """Generate predictions with a logistic regression model using a for-loop.

    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)

    Returns:
        y: probabilities of the positive class, shape (N)
    """
    # TODO
    pass

### Good - vectorization

Generate predictions with a logistic regression model using vectorized operations.

In [ ]:
def predict_vectorized(X, w, b):
    """Generate predictions with a logistic regression model using vectorized operations.

    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)

    Returns:
        y: probabilies of the positive class, shape (N)
    """
    # TODO
    pass

### Make sure that both variants produce the same results

In [ ]:
results_for_loop = predict_for_loop(X, w, b)
results_vectorized = predict_vectorized(X, w, b)

Are the results the same?

In [ ]:
# TODO

What is the norm of the difference?

In [ ]:
# TODO

Are they close enough?

In [ ]:
# TODO

### Compare the runtime of two variants

In [ ]:
%%timeit
predict_for_loop(X, w, b)

In [ ]:
%%timeit
predict_vectorized(X, w, b)

## 1.2. K-nearest neighbors
A more complicated task: compute the matrix of pairwise distances.

Given a data matrix `X` of size `[N, D]`, compute the matrix `dist` of pairwise distances of size `[N, N]`, where `dist[i, j] = l2_distance(X[i], X[j])`.

The L2 distance is:

$$
d_2(a, b) = \sqrt{\sum_{i=1}^d (a_i - b_i)^2}
$$

### Bad - for loops

In [ ]:
def l2_distance(x, y):
    """Compute Euclidean distance between two vectors."""
    # TODO
    pass

In [ ]:
def distances_for_loop(X):
    """Compute pairwise distances between all instances (for loop version).

    Args:
        X: data matrix, shape (N, D)

    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    # TODO
    pass

In [ ]:
# compute pairwise distances using for loops
dist1 = distances_for_loop(X)

### Good - vectorization

How can we compute all the distances in a vectorized way?

Start with a simpler example.

In [ ]:
x = np.arange(5, dtype=np.float64)
x

Use `numpy` broadcasting to compute the matrix of pairwise distances in a vectorized way. We achieve this by adding a new axis to `x` using `np.newaxis`.

In [ ]:
# TODO

In [ ]:
# TODO

In [ ]:
# TODO

The same result can be achieved using `None` indexing.

In [ ]:
# TODO

In [ ]:
# TODO

In [ ]:
def distances_vectorized(X):
    """Compute pairwise distances between all instances (vectorized version).

    Args:
        X: data matrix, shape (N, D)

    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    # TODO
    pass

In [ ]:
# compute pairwise distances using vectorized operations
dist2 = distances_vectorized(X)

### Make sure that both variants produce the same results

In [ ]:
np.allclose(dist1, dist2)

### Best - library function

In [ ]:
from scipy.spatial.distance import cdist, pdist, squareform

dist3 = cdist(X, X)
dist4 = squareform(pdist(X))

Make sure that both variants produce the same results

In [ ]:
# Use np.allclose to compare
np.allclose(dist2, dist3)

### Compare the runtime

In [ ]:
%%timeit
distances_for_loop(X)

In [ ]:
%%timeit
distances_vectorized(X)

In [ ]:
%%timeit
cdist(X, X)

In [ ]:
%%timeit
squareform(pdist(X))

## Lessons:
1. For-loops are extremely slow! Avoid them whenever possible.
2. A better alternative - use matrix operations & broadcasting
3. An even better alternative - use library functions (if they are available).
4. Implementations with for-loops can be useful for debugging vectorized code.

# 2. Numerical stability
Typically, GPUs use single precision (32bit) floating point numbers (in some cases even half precision / 16bit). This significantly speeds ups the computations, but also makes numerical issues a lot more likely. 
Because of this we always have to be extremely careful to implement our code in a numerically stable way.

Most commonly, numerical issues occur when dealing with `log` and `exp` functions (e.g. when computing cross-entropy of a categorical distribution) and `sqrt` for values close to zero (e.g. when computing standard deviations or normalizing the $L_2$ norm).

In [ ]:
np.finfo(np.float64), np.finfo(np.float32), np.finfo(np.float16)

## 2.1. Avoiding numerical overflow (exploding `exp`)

Softmax function $f : \mathbb{R}^D \to \Delta^{D - 1}$ converts a vector $\mathbf{x} \in \mathbb{R}^D$ into a vector of probabilities.

$$f(\mathbf{x})_j = \frac{\exp(x_j)}{\sum_{d=1}^{D} \exp(x_d)}$$

In [ ]:
def softmax_unstable(logits):
    # TODO
    pass

Apply the softmax function to the following vector.

In [ ]:
x = np.linspace(0.0, 4.0, 5).astype(np.float32)
x

In [ ]:
softmax_unstable(x)

Now apply it to the following vector

In [ ]:
x = np.linspace(50.0, 90.0, 5).astype(np.float32)
x

In [ ]:
softmax_unstable(x)

### How to avoid the explosion?

Shift the values by a constant $C$.

$$f(\mathbf{x})_j = \frac{\exp(x_j - C)}{\sum_{d=1}^{D} \exp(x_d - C)}$$

In [ ]:
def softmax_stable(logits):
    """Compute softmax values for each sets of scores in logits."""
    # TODO
    pass

In [ ]:
x = np.linspace(50.0, 90.0, 5).astype(np.float64)

In [ ]:
softmax_unstable(x)

## 2.2. Working in the log-space / simplifying the expressions

Binary cross entropy (BCE) loss for a logistic regression model (corresponds to negative log-likelihood of a Bernoulli model)

$$\log p(\mathbf{y} \mid \mathbf{X}, \mathbf{w}, b) = -\sum_{i=1}^{N} y_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) + (1 - y_i) \log (1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b))$$


Implement the BCE computation.

In [ ]:
def sigmoid(t):
    # TODO
    pass


def binary_cross_entropy_unstable(scores, labels):
    """Compute binary cross-entropy loss for one sample."""
    # TODO
    pass

In [ ]:
x = np.array([[20.0, 20.0]])  # [1, 2]
w = np.array([[1.0, 1.0]])  # [1, 2]
y = np.array([1.0])  # [1,]

# 1. compute logits
# TODO

# 2. compute loss
# TODO

Try to simplify the BCE loss as much as possible

In [ ]:
def binary_cross_entropy_stable(scores, labels):
    # TODO
    pass

In [ ]:
# 1. compute logits
# TODO

# 2. compute loss
# TODO

## 2.3. Loss of numerical precision

Implement the log sigmoid function 

$$f(x) = \log \sigma(x) = \log \left(\frac{1}{1 + \exp(-x)}\right)$$

In [ ]:
def log_sigmoid_unstable(x):
    # TODO
    pass

`float32` has much lower "resolution" than `float64`

In [ ]:
x = np.linspace(0, 30, 11).astype(np.float32)
x, log_sigmoid_unstable(x)

In [ ]:
x = np.linspace(0, 30, 11).astype(np.float64)
log_sigmoid_unstable(x)

Implement the log-sigmoid function in a numerically stable way

In [ ]:
def log_sigmoid_stable(x):
    # TODO
    pass

In [ ]:
x = np.linspace(0, 30, 11).astype(np.float32)
log_sigmoid_stable(x)

Relevant functions: `np.log1p`, `np.expm1`, `scipy.special.logsumexp`, `scipy.special.softmax` -- these are also implemented in all major deep learning frameworks.

## Lessons:
1. Be especially careful when working with `log` and `exp` functions in **single precision** floating point arithmetics
2. Work in the log-space when possible
3. Use numerically stable library functions when available